# Coursework 1

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

## Verification Data

Process the data using library functions for verification with the custom Hadoop implementation.

### Data Load

In [2]:
data_cols = [0, 1, 2, 8]
data_path = 'data/raw/200704hourly.txt'
df_weather = pd.read_csv(data_path, usecols=data_cols)

# remove spaces from the column names
df_weather.columns = df_weather.columns.str.replace(' ', '')

df_weather.shape

/home/jfoul001/.conda/envs/dsm010/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1338620, 4)

In [3]:
df_weather.sort_values(by='YearMonthDay', inplace=True)
df_weather.reset_index(drop=True, inplace=True)
df_weather.head()

,WbanNumber,YearMonthDay,Time,DryBulbTemp
0,3011,20070401,50,32
1,53969,20070401,1845,-
2,53969,20070401,1945,-
3,53969,20070401,2045,-
4,53969,20070401,2145,-


In [4]:
# replace '-' with empty NaN
df_weather_clean = df_weather.replace('-', np.NaN)
df_weather_clean[df_weather_clean.DryBulbTemp.isna()]

,WbanNumber,YearMonthDay,Time,DryBulbTemp
1,53969,20070401,1845,NaN
2,53969,20070401,1945,NaN
3,53969,20070401,2045,NaN
4,53969,20070401,2145,NaN
5,53969,20070401,2245,NaN
...,...,...,...,...
1338575,13902,20070430,923,NaN
1338576,13902,20070430,911,NaN
1338578,13902,20070430,835,NaN
1338580,13902,20070430,745,NaN


#### Remove Null Values

In [5]:
# drop null values
df_weather_clean = df_weather_clean.dropna()
df_weather_clean.YearMonthDay = df_weather_clean.YearMonthDay.astype('str')
df_weather_clean.DryBulbTemp = df_weather_clean.DryBulbTemp.astype('int')
df_weather_clean.shape

(1030138, 4)

In [6]:
df_weather_clean.describe()

,WbanNumber,DryBulbTemp
count,1.030138e+06,1.030138e+06
mean,3.734065e+04,5.167273e+01
std,3.395600e+04,1.626283e+01
min,3.011000e+03,-6.700000e+01
25%,1.290900e+04,4.000000e+01
50%,2.319000e+04,5.200000e+01
75%,5.483400e+04,6.400000e+01
max,9.499900e+04,1.120000e+02


#### Group By Day

In [7]:
df_weather_clean.query('YearMonthDay == "20070401"').DryBulbTemp.min()

-13

In [8]:
df_weather_grouped = df_weather_clean[['YearMonthDay', 'DryBulbTemp']].groupby(by='YearMonthDay')
df_weather_grouped
df_weather_grouped.agg(['max', 'min', 'mean', 'median', 'var']).reset_index()

YearMonthDay DryBulbTemp                                  
                        max min       mean median         var
0      20070401          92 -13  53.719618   54.0  220.167678
1      20070402          93 -13  55.066786   55.0  277.349574
2      20070403          93  -6  53.091015   54.0  347.583227
3      20070404          93 -15  45.994100   44.0  343.829264
4      20070405          94 -17  42.085650   39.0  293.334927
5      20070406          95  -5  39.700296   36.0  289.050116
6      20070407          96  -2  37.067186   34.0  228.100571
7      20070408          88  -6  39.363777   37.0  195.795372
8      20070409          89  -9  42.939378   42.0  184.965897
9      20070410          89 -15  45.496094   44.0  208.008217
10     20070411          96 -14  46.592362   45.0  234.703139
11     20070412          91 -16  46.828315   43.0  234.346236
12     20070413          98 -16  47.318374   45.0  222.475720
13     20070414          93 -11  47.918045   46.0  202.528936
14     20070415          90 -12  48.799602   48.0  181.024246
15     20070416          90 -13  50.581011   50.0  174.448033
16     20070417          90 -15  52.417287   52.0  172.907204
17     20070418          94   4  51.786859   51.0  153.899725
18     20070419          92  -4  52.154426   52.0  173.132032
19     20070420          93 -15  55.286874   56.0  187.983276
20     20070421          96 -11  58.056748   59.0  192.818167
21     20070422          89   1  59.943662   61.0  176.934579
22     20070423         112   1  60.829596   63.0  172.171014
23     20070424          99  12  59.212600   60.0  178.776399
24     20070425          96   3  56.825323   55.0  187.520616
25     20070426          96   0  56.331990   55.0  170.773712
26     20070427         102  -1  58.420641   58.0  178.732927
27     20070428         106 -11  61.101433   61.0  182.559690
28     20070429         100  -4  63.367894   63.0  195.306503
29     20070430         101 -67  63.332697   63.0  208.377827

## Mapper

In [9]:
class Mapper():
    """
    This class implements the logic for the mapping functionality.
    """
    def map_input(self, input_stream) -> list:
        """
        Map each item in the input stream to the output. The output is written to stdout.

        Parameters
        ----------
        input_stream : iterable
            The input stream to process.
        """
        output = []

        for item in input_stream:
            mapped_item = self.map(item)
            if mapped_item:
                print(mapped_item)

        return output

    def map(self, item:str) -> str:
        """
        Map the input string to the day the observations if for (key), and the observation temperature (value).

        Parameters
        ----------
        item : str
            The observation data to perform the mapping for.

        Returns
        -------
        output : dict
            A comma seperated string containing the `day` in the format YYYYMMDD and the 'temperature` as an integer.
        """
        # ignore the file headers
        if item.startswith('Wban Number'):
            return None

        # tokenize the input line
        tokens = item.split(',')

        # get the day value
        day_value = tokens[1].strip()

        # get the temperature
        temperature_value = tokens[8].strip()
        
        # do not process empty temperature values
        if temperature_value == '-':
            return None
        else:
            temperature_value = int(temperature_value)

        # return the key and value as a comma seperated string
        return '%s,%s' % (day_value, temperature_value)

# test the mapper with a text file
with open('data/raw/sample.txt') as input_file:
    mapper = Mapper()
    mapper.map_input(input_file)

20070401,32
20070401,32
20070401,32
20070403,34
20070401,34
20070401,32
20070402,34
20070401,37
20070401,41
20070401,45
20070401,50
20070401,52
20070402,55
20070403,-54
20070402,54
20070402,54
20070402,52
20070402,50
20070402,46
20070402,45


In [10]:
sample_path = data_path #'data/raw/sample.txt
with open(sample_path) as input_file:
    mapper = Mapper()
    output = list(map(mapper.map, input_file))

output

[None,
 '20070401,32',
 '20070401,32',
 '20070401,32',
 '20070401,34',
 '20070401,34',
 '20070401,32',
 '20070401,34',
 '20070401,37',
 '20070401,41',
 '20070401,45',
 '20070401,46',
 '20070401,50',
 '20070401,52',
 '20070401,55',
 '20070401,54',
 '20070401,54',
 '20070401,54',
 '20070401,52',
 '20070401,50',
 '20070401,46',
 '20070401,45',
 '20070401,43',
 '20070401,43',
 '20070401,43',
 '20070402,46',
 '20070402,45',
 '20070402,46',
 '20070402,45',
 '20070402,43',
 '20070402,41',
 '20070402,43',
 '20070402,45',
 '20070402,46',
 '20070402,50',
 '20070402,52',
 '20070402,54',
 '20070402,55',
 '20070402,54',
 '20070402,54',
 '20070402,54',
 '20070402,52',
 '20070402,52',
 '20070402,50',
 '20070402,50',
 '20070402,45',
 '20070402,45',
 '20070402,45',
 '20070402,43',
 '20070403,41',
 '20070403,37',
 '20070403,36',
 '20070403,36',
 '20070403,36',
 '20070403,34',
 '20070403,36',
 '20070403,41',
 '20070403,48',
 '20070403,52',
 '20070403,54',
 '20070403,55',
 '20070403,55',
 '20070403,55',
 

## Reducer

In [33]:
class ReducerValues():
    """
    Helper class to hold the values calculated by the reducer
    """
    def __init__(self, day:str) -> None:
        self.day = day
        self.max = None
        self.min = None
        self.values = []

    def add_value(self, value:int) -> None:
        """
        Add a value to the value collection
        """
        # add the value to the collection
        self.values.append(value)

        # update the max and min values
        if self.max is None:
            # initialize the max and min values
            self.max = value
            self.min = value
        else:
            if value > self.max: self.max = value
            if value < self.min: self.min = value

    def print_output(self) -> None:
        """
        Print the day values to stdout.
        """
        print('%s,%d,%d' % (
            self.day, 
            self.max, 
            self.min))

class Reducer():
    """
    This class contains the logic to summarize the temperature observations by day.
    """
    def reduce_input(self, input_stream):
        """
        Reduce the output from the mapper to calculate the max, min, mean, median, and variance per day.

        Parameters
        ----------
        input_stream : iterable
            The input stream to process.

        """
        current_day = ReducerValues(None)

        for item in input_stream:
            if item:
                # get the day and temperature value
                day_value, temperature_value = item.split(',')
                temperature_value = int(temperature_value)

                if current_day.day == day_value:
                    current_day.add_value(temperature_value)
                else:
                    # if the current day exist show the output
                    if current_day.day:
                        current_day.print_output()

                    # the current day have changed, create the new day
                    current_day = ReducerValues(day_value)
                    current_day.add_value(temperature_value)

        # print the last day processed
        if current_day.day == day_value:
            current_day.print_output()

with open(data_path) as input_file:
    mapper = Mapper()
    reducer = Reducer()
    
    # perform the mapping task
    mapper_output = map(mapper.map, input_file)

    # remove null values and sort the mapped output
    mapper_output = list(filter(None, mapper_output))
    mapper_output.sort()
    #print('len:', len(mapper_output))

    # perform the reduce function
    reducer.reduce_input(mapper_output)

20070401,92,-13
20070402,93,-13
20070403,93,-6
20070404,93,-15
20070405,94,-17
20070406,95,-5
20070407,96,-2
20070408,88,-6
20070409,89,-9
20070410,89,-15
20070411,96,-14
20070412,91,-16
20070413,98,-16
20070414,93,-11
20070415,90,-12
20070416,90,-13
20070417,90,-15
20070418,94,4
20070419,92,-4
20070420,93,-15
20070421,96,-11
20070422,89,1
20070423,112,1
20070424,99,12
20070425,96,3
20070426,96,0
20070427,102,-1
20070428,106,-11
20070429,100,-4
20070430,101,-67
